In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from os.path import join as pjoin
from scipy.io import savemat, loadmat
from cxy_hcp_ffa.lib.predefine import net2label_cole

proj_dir = '/nfs/t3/workingshop/chenxiayu/study/FFA_pattern'
anal_dir = pjoin(proj_dir, 'analysis/s2/1080_fROI/refined_with_Kevin')
work_dir = '/nfs/s2/userhome/chenxiayu/workingdir/test/ffa_fig'

# save out myelin and thickness

In [5]:
meas_name = 'myelin'  # myelin, thickness
gids = [0, 1, 2]
hemis = ['lh', 'rh']
rois = ['pFus-face', 'mFus-face']
src_file = pjoin(anal_dir, 'structure/rois_v3_{hemi}_{meas}.pkl')
gid_file = pjoin(anal_dir, 'grouping/group_id_{hemi}.npy')
out_file = pjoin(work_dir, f'{meas_name}.mat')

hemi2data = {}
hemi2gid_vec = {}
for hemi in hemis:
    hemi2data[hemi] = pkl.load(open(src_file.format(hemi=hemi, meas=meas_name), 'rb'))
    hemi2gid_vec[hemi] = np.load(gid_file.format(hemi=hemi))

out_data = {}
for gid in gids:
    for roi in rois:
        for hemi in hemis:
            k = f"{hemi}_{roi.split('-')[0]}_{gid}"
            data = hemi2data[hemi]
            gid_vec = hemi2gid_vec[hemi]
            v = np.ones_like(gid_vec, np.float64) * np.nan
            gid_idx_vec = gid_vec == gid
            roi_idx = data['roi'].index(roi)
            v[gid_idx_vec] = data['meas'][roi_idx][gid_idx_vec]
            out_data[k] = v

savemat(out_file, out_data)

# save out RSFC

In [4]:
gids = [0, 1, 2]
hemis = ('lh', 'rh')
seed_names = ['pFus-face', 'mFus-face']
src_file = pjoin(anal_dir, 'rfMRI/rsfc_individual2Cole_{hemi}.pkl')
gid_file = pjoin(anal_dir, 'grouping/group_id_{hemi}.npy')
out_file = pjoin(work_dir, 'rsfc.mat')

n_hemi = len(hemis)
hemi2data = {}
hemi2gid_vec = {}
for hemi in hemis:
    hemi2data[hemi] = pkl.load(open(src_file.format(hemi=hemi), 'rb'))
    hemi2gid_vec[hemi] = np.load(gid_file.format(hemi=hemi))
n_seed = len(seed_names)

label2net_cole = {}
for k, v in net2label_cole.items():
    label2net_cole[v] = k
assert hemi2data['lh']['trg_label'] == hemi2data['rh']['trg_label']
trg_labels = hemi2data['lh']['trg_label']
trg_names = [label2net_cole[_] for _ in trg_labels]
n_trg = len(trg_labels)

out_data = {
    'data_shape': 'n_targets x n_subjects',
    'target_label': trg_labels,
    'target_name': np.array(trg_names, np.object_)
}
for gid in gids:
    for seed_name in seed_names:
        for hemi_idx, hemi in enumerate(hemis):
            k = f"{hemi}_{seed_name.split('-')[0]}_{gid}"
            data = hemi2data[hemi]
            gid_vec = hemi2gid_vec[hemi]
            gid_idx_vec = gid_vec == gid
            v = np.ones_like(data[seed_name]) * np.nan
            v[gid_idx_vec] = data[seed_name][gid_idx_vec]
            out_data[k] = v.T

savemat(out_file, out_data)

# save out cortical distances per group

In [4]:
method = 'AP_gap-geo'  # 'peak', 'min', 'AP_gap-y', 'AP_gap-geo'
hemis = ('lh', 'rh')
gids = (1, 2)
data_file = pjoin(anal_dir, f'gdist_{method}.csv')
gid_file = pjoin(anal_dir, 'grouping/group_id_{hemi}.npy')
out_file = pjoin(work_dir, f'cortial_distance_{method}.mat')

df = pd.read_csv(data_file)
out_data = {}
for hemi in hemis:
    gid_vec = np.load(gid_file.format(hemi=hemi))
    for gid in gids:
        k = f'{hemi}_{gid}'
        col = f'{hemi}_pFus-mFus'
        gid_idx_vec = gid_vec == gid
        v = np.ones_like(gid_idx_vec, np.float64) * np.nan
        v[gid_idx_vec] = np.array(df[col])[gid_idx_vec] / 10
        out_data[k] = v

savemat(out_file, out_data)